In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import PolynomialFeatures

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Caricare i dati di training, i dati di test, e verificare il # di osservazioni e il # di caratteristiche per ciascuno dei due
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Shape di addestramento: ', app_train.shape)
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Shape di addestramento: ', app_test.shape)

In [ ]:
# Create a label encoder object
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

def mylabelencoder(df):
    le_count = 0
    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 3:
                df[col] = df[col].fillna("")
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])

                # Keep track of how many columns were label encoded
                le_count += 1            
    print('%d columns were label encoded.' % le_count)

In [ ]:
# Controlla che codifichi lo stesso numero di colonne su train e su test
mylabelencoder(app_train)
mylabelencoder(app_test)

In [ ]:
# One Hot Encoding delle altre feature categoriche
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [ ]:
# Riallineamento delle colonne
# Cerca le colonne differenti tra i due set, togliendo dal train la target per cercare le differenze
colset_train = set(app_train.columns)
colset_test = set(app_test.columns)
colset_train.discard('TARGET')
colset_train.difference(colset_test)

In [ ]:
for k in list(colset_train.difference(colset_test)):
    app_test[k] = 0 # Default a 0 perché se le colonne non appaiono in test sono sempre non-hot
    
print(app_train.shape)
print(app_test.shape)
set(app_train.columns).difference(set(app_test.columns))

# Test ha una colonna in meno perché non contiene TARGET

In [ ]:
# Analisi degli outlier
app_train.dtypes.value_counts()

In [ ]:
(app_train["DAYS_BIRTH"]/-365).describe()

In [ ]:
(app_train["DAYS_EMPLOYED"]/365).describe()

In [ ]:
app_train["DAYS_EMPLOYED"].plot(kind="hist")

In [ ]:
 app_train["DAYS_EMPLOYED"] == 365243

In [ ]:
# Identificare e filtrare le osservazioni che hanno la caratt. DAYS EMPLOYED con un valore outlier
# Per caso, l'outlier è legato al fatto che il prestito è andato in default? In che proporzione di occasioni?
anomalies = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED_ANOM'] = anomalies
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
(app_train['DAYS_EMPLOYED']/365).plot.hist(title = 'Days Employment Histogram');


In [ ]:
app_train["DAYS_EMPLOYED_ANOM"].value_counts()

In [ ]:
#import seaborn as sns
#sns.pairplot(app_train.iloc[:,:10])
# Plotting as desired
# app_train.iloc[:,27:70].plot.hist(subplots=True, legend=False)
# Analisi delle collinearità
#correlations = app_train.corr()
#correlations
#correlations["TARGET"].sort_values().head(10)
#correlations["TARGET"].sort_values().tail(10)
#app_train['DAYS_BIRTH'].corr(app_train['TARGET'])
#app_train['DAYS_BIRTH']/-365).corr(app_train['TARGET'])
#app_train['DAYS_BIRTH']/-365).corr(app_train['TARGET'])

In [ ]:
##import seaborn as sns
#import matplotlib.pyplot as plt

#plt.style.use('fivethirtyeight')
#plt.figure(figsize = (10, 8))

#sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'DAYS_BIRTH'] / -365, label = 'target == 0' )
#sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'DAYS_BIRTH'] / -365, label = 'target == 1' )

# Metti la legenda
#plt.legend()


In [ ]:
# Creare dei kernel plot su EXT_SOURCE(1,2,3) rispetto a TARGET per valutarne la "quantità di segnale"

#sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'EXT_SOURCE_1'], label = 'target == 0' )
#sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'EXT_SOURCE_1'], label = 'target == 1' )

# Metti la legenda
#plt.legend()

In [ ]:
# Creare dei kernel plot su EXT_SOURCE(1,2,3) rispetto a TARGET per valutarne la "quantità di segnale"

#sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'EXT_SOURCE_2'], label = 'target == 0' )
#sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'EXT_SOURCE_2'], label = 'target == 1' )

# Metti la legenda
#plt.legend()

In [ ]:
# Creare dei kernel plot su EXT_SOURCE(1,2,3) rispetto a TARGET per valutarne la "quantità di segnale"

#sns.kdeplot(app_train.loc[app_train['TARGET'] == 0, 'EXT_SOURCE_3'], label = 'target == 0' )
#sns.kdeplot(app_train.loc[app_train['TARGET'] == 1, 'EXT_SOURCE_3'], label = 'target == 1' )

# Metti la legenda
#plt.legend()

In [ ]:
# Extract the EXT_SOURCE variables and show correlations
#ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
#ext_data_corrs = ext_data.corr()
#ext_data_corrs

In [ ]:
#plt.figure(figsize = (8, 6))

# Heatmap of correlations
#sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
#plt.title('Correlation Heatmap');

In [ ]:
# Feature Engineering
# Combinazioni polinomiali delle caratteristiche
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html
#poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
#poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
#poly_target = app_train['TARGET']

In [ ]:
# Imputazione dei valori mancanti
# https://scikit-learn.org/stable/modules/impute.html
#poly_features['EXT_SOURCE_1'].plot(kind="hist")

In [ ]:
#poly_features['EXT_SOURCE_1'].isnull().value_counts()

In [ ]:
# Inseriamo NaN al posto di null
#poly_features['EXT_SOURCE_1'] = poly_features['EXT_SOURCE_1'].fillna(np.nan)
#poly_features

In [ ]:
# Uso di SimpleImputer per imputazione automatica di valori tramite una funzione nelle celle vuote
#import numpy as np
#from sklearn.impute import SimpleImputer
#imp = SimpleImputer(missing_values=np.nan, strategy='median')
#poly_features_imputed = imp.fit_transform(poly_features)
#poly_features_imputed_df = pd.DataFrame(poly_features_imputed, columns=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])

In [ ]:
#poly_features_test_imputed = imp.fit_transform(poly_features_test)
#poly_features_test_imputed_df = pd.DataFrame(poly_features_test_imputed, columns=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])

In [ ]:
# Tappati tutti i buchi con le mediane
#poly_features_imputed_df.isnull().value_counts()
#poly_features_test_imputed_df.isnull().value_counts()

In [ ]:
# Calcola combinazioni polinomiali delle feature del dataframe Df
#from sklearn.preprocessing import PolynomialFeatures
#def poly_transformer(df):
 #   poly_transformer = PolynomialFeatures(degree = 3)
  #  poly_transformer.fit(df)
   # poly_features_df = pd.DataFrame(poly_transformer.transform(df), columns=poly_transformer.get_feature_names(df.columns))
    #poly_features_df = poly_features_df.drop(labels='1', axis=1)
    #return poly_features_df

In [ ]:
# Ora finalmente posso usare le polynomial features
#from sklearn.preprocessing import PolynomialFeatures
#poly_transformer = PolynomialFeatures(degree = 3)
#poly_transformer.fit(poly_features_imputed_df)

In [ ]:
#poly_transformer.transform(poly_features_imputed_df)

In [ ]:
#poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])

In [ ]:
#poly_features_df = pd.DataFrame(poly_transformer.transform(poly_features_imputed_df), columns=poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))

In [ ]:
#poly_features_df

In [ ]:
#poly_features_df = poly_features_df.drop(labels='1', axis=1)

In [ ]:
#poly_features_df['TARGET'] = poly_target

In [ ]:
# Vediamo ora se rispetto alle sole features predominanti di base, quelle costruite in polinomiali risultano maggiormente correlate al target
#poly_collinears = poly_features_df.corr()

In [ ]:
#poly_collinears['TARGET'].sort_values().head(5)

In [ ]:
#poly_collinears['TARGET'].sort_values().tail(5)

In [ ]:
#poly_features_test_df = pd.DataFrame(poly_transformer.transform(poly_features_test_imputed_df), columns=poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']))

In [ ]:
#poly_features_test_df = poly_features_test_df.drop(labels='1', axis=1)

In [ ]:
#prova_poly_features

In [ ]:
# Provo la mia funzione
#prova_poly_features = poly_features_imputed_df.copy() # copia per separare i df nella trasformazione
#pippo = poly_transformer(prova_poly_features)

In [ ]:
# Sistemare la poly transformation in una funzione
# Creare delle caratteristiche derivate ("in modo business")


In [ ]:
# Creo una copia per valore e non per riferimento dei dati di train e di test
#app_train_domain = app_train.copy()
#app_test_domain = app_test.copy()

In [ ]:
# Funzione che calcola nuove caratteristiche di dominio
#def calculate_domain_data(df):    
 #   df['CREDIT_INCOME_PERCENT'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
  #  df['ANNUITY_INCOME_PERCENT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
   # df['CREDIT_TERM'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    #df['DAYS_EMPLOYED_PERCENT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

In [ ]:
#calculate_domain_data(app_train_domain)

In [ ]:
#app_train_domain

In [ ]:
# Infine, provare con la visualizzazione kernel density a dimostrare se qualcuna di queste nuove caratteristiche create (sia polinominali, sia di business)
# ha un particolare potere "separatorio" rispetto al target
#def mykdeplot(df, field):
 #   sns.kdeplot(df.loc[app_train['TARGET'] == 0, field], label = 'target == 0' )
  #  sns.kdeplot(df.loc[app_train['TARGET'] == 1, field], label = 'target == 1' )
    
    # Metti la legenda
   # plt.legend()

In [ ]:
#mykdeplot(poly_features_df, 'EXT_SOURCE_2 EXT_SOURCE_3')

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
app_train_imputed = imp.fit_transform(app_train)
app_train_imputed_df = pd.DataFrame(app_train_imputed, columns=app_train.columns)

In [ ]:
app_test_imputed = imp.fit_transform(app_test)
app_test_imputed_df = pd.DataFrame(app_test_imputed, columns=app_test.columns)

In [ ]:
# # 2) Anziché fittare il modello su tutto il df "app_train", usare train_test_split per splittare app_train in app_train_train e app_train_test

from sklearn.model_selection import train_test_split
X=app_train_imputed_df.drop(['TARGET','DAYS_EMPLOYED_ANOM'], axis=1)
y=app_train_imputed_df['TARGET'].to_frame() 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
print(X_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C = 0.0001, solver='liblinear') 
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

In [ ]:
# uso il modello fittato per fare la previsione
log_reg.predict_proba(app_test_imputed_df)[:,1]

In [ ]:
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg.predict_proba(app_test_imputed_df)[:,1]

In [ ]:
submit

In [ ]:
submit.to_csv('submission.csv', index=False)

In [ ]:
#from sklearn import svm
#clf = svm.SVC(kernel='linear', C=1, probability=True).fit(X_train, y_train)
#clf.score(X_test, y_test)

In [ ]:
# 6) RIpetere usando RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train, y_train)
rf.score(X_test, y_test) 

In [ ]:
rf.predict_proba(app_test_imputed_df)[:,1]

In [ ]:
submitrf = app_test[['SK_ID_CURR']]
submitrf['TARGET'] = rf.predict_proba(app_test_imputed_df)[:,1]
submitrf.to_csv('submission.csv', index=False)


In [ ]:
# Extract feature importances
features=list(app_test_imputed_df.columns)
feature_importance_values = rf.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})
feature_importances = feature_importances.sort_values('importance',ascending=False)
feature_importances

In [ ]:
app_train_top=feature_importances[:10]
app_train_top

In [ ]:
app_test_top=app_test_imputed_df[['SK_ID_CURR','EXT_SOURCE_3','EXT_SOURCE_2','DAYS_BIRTH','CODE_GENDER','DAYS_EMPLOYED','REGION_RATING_CLIENT']]
app_test_top

In [ ]:
#seleziono solo caratteristiche più significative
app_train_top=app_train_imputed_df[['SK_ID_CURR','TARGET','EXT_SOURCE_3','EXT_SOURCE_2','DAYS_BIRTH','CODE_GENDER','DAYS_EMPLOYED','REGION_RATING_CLIENT']]
app_train_top


In [ ]:
from sklearn.model_selection import train_test_split
X=app_train_top.drop(['TARGET'], axis=1)
y=app_train_top['TARGET'].to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
#ripeto la random forest
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier(max_depth=4, random_state=0)
rf1.fit(X_train, y_train)
rf1.score(X_test, y_test)

In [ ]:
rf1.predict_proba(app_test_top)[:,1]

In [ ]:
submitrf1 = app_test[['SK_ID_CURR']]
submitrf1['TARGET'] = rf1.predict_proba(app_test_top)[:,1]
submitrf1['TARGET'] = rf1.predict_proba(app_test_top)[:,1]
submitrf1.to_csv('submission.csv', index=False)

In [ ]:
# Caricare i dati di bureau
#bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
#print('bureau: ', bureau.shape)
#bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
#print('bureau_balance: ', bureau_balance.shape)

In [ ]:
#mylabelencoder(bureau)
#mylabelencoder(bureau_balance)

In [ ]:
# One Hot Encoding delle altre feature categoriche
#bureau = pd.get_dummies(bureau)
#bureau_balance = pd.get_dummies(bureau_balance)

In [ ]:
# Analisi degli outlier
#bureau.dtypes.value_counts()
#bureau_balance.dtypes.value_counts()

In [ ]:
#import numpy as np
#from sklearn.impute import SimpleImputer
#imp = SimpleImputer(missing_values=np.nan, strategy='median')
#bureau_imputed = imp.fit_transform(bureau)
#bureau_imputed_df = pd.DataFrame(bureau_imputed, columns=bureau.columns)


In [ ]:
#bureau_balance_imputed = imp.fit_transform(bureau_balance)
#bureau_balance_imputed_df = pd.DataFrame(bureau_balance, columns=app_test.columns)

In [ ]:
#top_bureau=pd.merge(app_train_top,bureau_imputed_df,on='SK_ID_CURR',how='right')

In [ ]:
#app_train_top.info()

In [ ]:
#bureau_imputed_df.info()

In [ ]:
#from sklearn.model_selection import train_test_split
#X=top_bureau.drop(['TARGET'], axis=1)
#y=top_bureau['TARGET'].to_frame()
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
#print(X_test.shape)
#print(y_test.shape)

In [ ]:
#ripeto la random forest
#from sklearn.ensemble import RandomForestClassifier
#rf1 = RandomForestClassifier(max_depth=2, random_state=0)
#rf1.fit(X_train, y_train)
#rf1.score(X_test, y_test)